# Paso1: Seleccion y preparacion del Dataset

In [1]:
import pandas as pd
import requests
import os

# URLs de los archivos
urls = [
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-01.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-02.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-03.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-04.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-05.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-06.parquet"
]

# Función para descargar archivo
def download_file(url, filename):
    print(f"Descargando {filename}...")
    response = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"✅ {filename} descargado")

# Descargar todos los archivos
for url in urls:
    filename = url.split('/')[-1]  # Extrae el nombre del archivo
    if not os.path.exists(filename):
        download_file(url, filename)
    else:
        print(f"⏭️  {filename} ya existe, saltando...")

print("🎉 Todas las descargas completadas!")

Descargando yellow_tripdata_2016-01.parquet...
✅ yellow_tripdata_2016-01.parquet descargado
Descargando yellow_tripdata_2016-02.parquet...
✅ yellow_tripdata_2016-02.parquet descargado
Descargando yellow_tripdata_2016-03.parquet...
✅ yellow_tripdata_2016-03.parquet descargado
Descargando yellow_tripdata_2016-04.parquet...
✅ yellow_tripdata_2016-04.parquet descargado
Descargando yellow_tripdata_2016-05.parquet...
✅ yellow_tripdata_2016-05.parquet descargado
Descargando yellow_tripdata_2016-06.parquet...
✅ yellow_tripdata_2016-06.parquet descargado
🎉 Todas las descargas completadas!


Verificación de Archivos Descargados

In [2]:
import os
import pandas as pd

# Verificar archivos descargados
archivos_esperados = [
    "yellow_tripdata_2016-01.parquet",
    "yellow_tripdata_2016-02.parquet", 
    "yellow_tripdata_2016-03.parquet",
    "yellow_tripdata_2016-04.parquet",
    "yellow_tripdata_2016-05.parquet",
    "yellow_tripdata_2016-06.parquet"
]

print("=== VERIFICACIÓN DE ARCHIVOS ===")
for archivo in archivos_esperados:
    if os.path.exists(archivo):
        size_mb = os.path.getsize(archivo) / (1024*1024)
        print(f"✅ {archivo} - {size_mb:.1f} MB")
    else:
        print(f"❌ {archivo} - NO ENCONTRADO")

=== VERIFICACIÓN DE ARCHIVOS ===
✅ yellow_tripdata_2016-01.parquet - 144.2 MB
✅ yellow_tripdata_2016-02.parquet - 150.8 MB
✅ yellow_tripdata_2016-03.parquet - 162.1 MB
✅ yellow_tripdata_2016-04.parquet - 157.9 MB
✅ yellow_tripdata_2016-05.parquet - 158.1 MB
✅ yellow_tripdata_2016-06.parquet - 149.0 MB


Combinación de Archivos

In [1]:
import pandas as pd
import numpy as np

print("🎯 Procesando SOLO Enero 2016 (muestra representativa)")

# Cargar solo el primer archivo
df = pd.read_parquet("yellow_tripdata_2016-01.parquet")
print(f"Registros originales: {len(df):,}")

# Tomar muestra aleatoria de 500k registros 
np.random.seed(42)  # Para reproducibilidad
sample_size = 500000

if len(df) > sample_size:
    df_sample = df.sample(n=sample_size, random_state=42)
    print(f"Muestra aleatoria: {len(df_sample):,} registros")
else:
    df_sample = df
    print(f"Dataset completo: {len(df_sample):,} registros")

# Verificar distribución temporal
print(f"Período: {df_sample['tpep_pickup_datetime'].min()} a {df_sample['tpep_pickup_datetime'].max()}")

🎯 Procesando SOLO Enero 2016 (muestra representativa)
Registros originales: 10,905,067
Muestra aleatoria: 500,000 registros
Período: 2016-01-01 00:00:00 a 2016-01-31 23:59:20


Creación de Variable Target y Limpieza

In [2]:
# Crear variable target: duración del viaje
print("⏱️  Calculando duración de viajes...")

df_sample['trip_duration'] = (
    df_sample['tpep_dropoff_datetime'] - df_sample['tpep_pickup_datetime']
).dt.total_seconds()

# Estadísticas iniciales de duración
print(f"Duración promedio: {df_sample['trip_duration'].mean()/60:.1f} minutos")
print(f"Duración mediana: {df_sample['trip_duration'].median()/60:.1f} minutos")
print(f"Duración mínima: {df_sample['trip_duration'].min()/60:.1f} minutos")
print(f"Duración máxima: {df_sample['trip_duration'].max()/60:.1f} minutos")

# Verificar valores problemáticos
invalid_durations = (df_sample['trip_duration'] <= 0).sum()
extreme_durations = (df_sample['trip_duration'] > 10800).sum()  # >3 horas
print(f"⚠️  Duraciones inválidas (≤0): {invalid_durations:,}")
print(f"⚠️  Duraciones extremas (>3h): {extreme_durations:,}")

⏱️  Calculando duración de viajes...
Duración promedio: 15.2 minutos
Duración mediana: 10.4 minutos
Duración mínima: 0.0 minutos
Duración máxima: 31113.0 minutos
⚠️  Duraciones inválidas (≤0): 549
⚠️  Duraciones extremas (>3h): 750


Verificar Nombres de Columnas

In [4]:
# Verificar estructura del dataset
print("🔍 VERIFICANDO ESTRUCTURA DEL DATASET")
print(f"Columnas disponibles: {list(df_sample.columns)}")
print(f"Total columnas: {len(df_sample.columns)}")
print("\nPrimeras 3 filas:")
print(df_sample.head(3))

🔍 VERIFICANDO ESTRUCTURA DEL DATASET
Columnas disponibles: ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee', 'trip_duration']
Total columnas: 20

Primeras 3 filas:
          VendorID tpep_pickup_datetime tpep_dropoff_datetime  \
10490098         1  2016-01-30 21:46:05   2016-01-30 22:10:42   
2864959          2  2016-01-09 11:08:21   2016-01-09 11:13:59   
2557844          2  2016-01-08 16:05:52   2016-01-08 16:26:48   

          passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
10490098                1           2.20           1                  N   
2864959                 2           1.01           1                  N   
2557844                 1           4.06           1                  N  

Descargar Archivo de Mapeo de Zonas (estoy aquiii)